In [1]:
import re
import camelot
import pickle
import pandas as pd

pickle_file_path="../output/caljobs_activity_codes_dictionary_camelot_output.pkl"

tbls = camelot.read_pdf("../sourcedata/caljobs_activity_codes_dictionary.pdf", pages="2-61")

with open(pickle_file_path, "wb") as handle:
    pickle.dump(tbls, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
with open(pickle_file_path, "rb") as handle:
    tbls_pkl = pickle.load(handle)

activity_code_list = []
activity_name_list = []
activity_description_list = []

for tbl in tbls_pkl:
    df = tbl.df.tail(-1)
    activity_code_list.extend(df[0].values.tolist())
    name_splits = df[1].str.split("\n")
    idx = 0
    for name_split in name_splits:
        name_split_copy = name_split.copy()
        for index, item in enumerate(name_split):
            if len(item.strip()) >= 3:
                activity_name_list.append(item.strip())
                name_split_copy.remove(item)
                activity_description_list.append(" ".join(name_split_copy))
                break

caljobs_act_code_dictionary = pd.DataFrame({"activity_code":activity_code_list, "activity_name": activity_name_list, "activity_description": activity_description_list})
caljobs_act_code_dictionary['activity_description'] = caljobs_act_code_dictionary['activity_description'].str.replace(r"\s*\*\s+", "", regex=True)
missing_140 = caljobs_act_code_dictionary.loc[141,'activity_name'] + " " + caljobs_act_code_dictionary.loc[141,'activity_description']
caljobs_act_code_dictionary.loc[140,'activity_description'] = missing_140
caljobs_act_code_dictionary = caljobs_act_code_dictionary.drop(index = 141, axis=0).reset_index(drop=True)
caljobs_act_code_dictionary["activity_originator"] = caljobs_act_code_dictionary["activity_code"].apply(lambda x: "System" if x[0].isnumeric() else "Employer" if x[0]=="E" else "Follow-up")

caljobs_act_code_dictionary.to_csv("../output/caljobs_activity_codes_dictionary.csv")